Tutorial은 아래 url을 전적으로 참고하였음

https://github.com/marcotcr/lime

- classification 위주로 작성
- multiclass classification 및 regression은 소스 코드가 조금 더 복잡함
- 대표적은 classification 문제인 iris 데이터셋을 binary classification으로 수정후 결과를 생성하였음

In [ ]:
import sklearn
import pandas as pd
from sklearn.preprocessing import LabelBinarizer
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from datetime import datetime

In [ ]:
# 학습데이터 csv파일 경로 정의
datatag = 'bikedemand'
train_data_path = 'data/%s_train.csv'%(datatag)
test_data_path  = 'data/%s_test.csv'%(datatag)

In [ ]:
# CSV파일을 pandas dataframe으로 불러오기

df_train = pd.read_csv(train_data_path)
df_test = pd.read_csv(test_data_path) # BLIND DATA, FOR SUBMISSION

# PRINT WHOLE COLUMNS
print(df_train.columns)
print(df_test.columns)
print(df_train)


# DEFINE FEATURE, TARGET COLUMNS
feature = ['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed']
target = ['target']


print(df_train.weather.drop_duplicates())

In [ ]:
# REGRESSION 데이터셋인 BIKEDEMAND를 간단하게 CLASSIFICATION 데이터셋으로 변경한다
df_train['target'] = df_train['count']>100

print(df_train)

In [ ]:
# WITHOUT PREPROCESSING
train_x, test_x, train_y, test_y = train_test_split(df_train[feature], 
                                                    df_train[target], 
                                                    random_state=0, 
                                                    test_size=0.25)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier().fit(train_x, train_y)
# print(inspect.getsource(eachRegressor().fit))
# VALIDATION (SCORE)
print(gbm)
print("TRAIN DATASET SCORE:", gbm.score(train_x, train_y))
print("TEST  DATASET SCORE:", gbm.score(test_x, test_y))
print("TEST  DATASET PREDICTION", gbm.predict(test_x[:5]))


# ONE LINE PARSING LIST
print("TEST  DATASET PREDICTION", [round(float(prd),2) for prd in  gbm.predict(test_x[:40])])
print('-'*100)

## 이상 임의의 bike demand 데이터를 이용해서 GBM CLASSIFICATION 모델 생성하였음 

이하 Lime explainer 생성

In [ ]:
# Tabular 데이터를 사용하여 머신러닝 모델을 구축하였으므로, LimeTabularExplainer을 사용한다

from lime.lime_tabular import LimeTabularExplainer
# from lime.lime_image import LimeImageExplainer



In [ ]:
# define predict function and run predict using sample_dataframe

def predict_fn(df):
	return gbm.predict_proba(df)  # predict 대신 predict_proba를 사용한다

# test
prediction = predict_fn(test_x)
print(prediction)

gbm.predict(test_x)

In [ ]:
LimeTabularExplainer?

In [ ]:
print(train_x.dtypes)

In [ ]:
explainer = LimeTabularExplainer(training_data=train_x.values, 
                                 mode='classification', 
                                 feature_names=train_x.columns, 
                                 categorical_features=[0,1,2,3],
                                 categorical_names=['season', 'holiday', 'workingday', 'weather'],
                                 class_names=['False','True'])

In [ ]:
explanation = explainer.explain_instance(test_x.values[0], 
                                         predict_fn, 
                                         num_features=len(test_x.columns))

In [ ]:
print(test_y.values[0])

explanation.show_in_notebook(show_table=True)

In [ ]:
explanation.as_list()  # representaion, weight

In [ ]:
explanation??

In [ ]:
## Explanation에 대한 해석은 아래 링크를 참조
## https://github.com/marcotcr/lime